In [1]:
import numpy as np

In [2]:
def TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return, v_nominal):
    
     initial_port_value = np.dot(n_bonds, initial_prices)
     final_port_value = np.dot(n_bonds, final_prices)
     coupon_pmt = np.sum(n_bonds * coupons/t * v_nominal)
     total_return = (final_port_value - initial_port_value) + coupon_pmt
     rate = fixed_rate/t * initial_port_value

     return print(f'${(rate - total_return if pay_total_return else total_return - rate)[0]:,.2f}')

In [4]:
n_bonds = np.array([104_000, 96_000])
coupons = np.array([0.087, 0.103])
initial_prices = np.array([96.45, 97.84])
final_prices = np.array([96.03, 97.63])
fixed_rate = np.array([0.075])
t = np.array([2])
pay_total_return = False
v_nominal = np.array([100, 100])

TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return, v_nominal)

$154,581.00
